In [ ]:
from typing import Dict
import pandas as pd
import numpy as np

ds = pd.read_json("dataset.json")

def preprocess_item(item: Dict):
  ls = []
  for a1, a2 in zip(item['correct answer'], item['correct answer']):
    for entry in item['incorrect answers']:
      ls.append(pd.Series(
          {
              'c_answer': a1,
              's_answer': entry,
              'score': 0.0,
          }
      ))
    ls.append(pd.Series(
        {
            'c_answer': a1,
            's_answer': a2,
            'score': 1.0,
        }
    ))
  return ls

def preprocess_data(data):
  to_df = []
  for _, item in data.iterrows():
    to_df += preprocess_item(item[0])
  return pd.DataFrame(to_df)

new_ds = preprocess_data(ds)

In [ ]:
from torch.utils.data import Dataset
from sentence_transformers import InputExample
class CollectedDataset(Dataset):
  def __init__(self, data):
    self.c_answers = new_ds.loc[:, "c_answer"]
    self.s_answers = new_ds.loc[:, "s_answer"]
    self.scores = new_ds.loc[:, "score"]

  def __getitem__(self, item):
    return InputExample(
        texts=[self.c_answers[item],self.s_answers[item]],
        label=float(self.scores[item])
        )

  def __len__(self):
    return len(self.c_answers)


collected_data = CollectedDataset(new_ds)
type(collected_data[0].label)

float

In [ ]:
model_name = "msmarco-distilbert-base-v4"
model_path = ""
batch_size = 32
epochs = 5

In [ ]:
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader

model = SentenceTransformer(model_name)
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
train_dataloader = DataLoader(collected_data, shuffle=True, batch_size=batch_size)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
model.save("last_trained")

In [ ]:
from sentence_transformers.util import cos_sim

entry = new_ds.iloc[53]
s1 = entry.loc["c_answer"]
s2 = entry.loc["s_answer"]
e1 = model.encode([s1],  convert_to_tensor=True)
e2 = model.encode([s2],  convert_to_tensor=True)
print(f"Correct Answer: {s1}\nStudent answer: {s2} ",
    "\nPredicted:", cos_sim(e1, e2), "Actual:", entry.loc["score"])

Correct Answer: Pharaoh Ramses XI
Student answer: Pharaoh Hatshepsut.  
Predicted: tensor([[0.5773]]) Actual: 0.0
